In [2]:
import sys
import os

toolkit_dir = os.path.abspath('./toolkit')
sys.path.append(toolkit_dir)

import numpy as np
import streamlit as st
from toolkit.adkp_queries import (
    query_project_summary,
    query_monthly_downloads_per_study
)
from toolkit.utils import get_data_from_snowflake
from toolkit.widgets import plot_download_sizes, plot_unique_users_trend

2024-08-29 11:42:55.724 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
# 1. Retrieve the data using your queries in queries.py
project_summary_df = get_data_from_snowflake(query_project_summary())

2024-08-29 11:42:56.076 
  command:

    streamlit run /Users/alinden/.pyenv/versions/3.10.9/envs/snowflake-streamlit-3.10.9/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-08-29 11:42:56.078 No runtime found, using MemoryCacheStorageManager


In [4]:
project_summary_df

,TOTAL_FILES,TOTAL_SIZE_IN_TB,STORAGE_COST_PER_YEAR
0,260523,811.82,229439.41


In [5]:
unique_users_df = get_data_from_snowflake(query_monthly_downloads_per_study())

In [5]:
unique_users_df

,STUDY,MONTH,NUMBER_OF_DOWNLOADS,NUMBER_OF_UNIQUE_USERS,GIB_DOWNLOADED
0,ROSMAP,2024-08-01,67132,149,100530.14
1,MIT_ROSMAP_Multiomics,2024-08-01,86228,55,134748.39
2,AMP-AD_DiverseCohorts,2024-08-01,35172,26,48145.17
3,MSBB,2024-08-01,3456,25,2821.03
4,rnaSeqReprocessing,2024-08-01,2362,19,19715.00
...,...,...,...,...,...
2417,APOEPSC,2022-01-01,2,1,0.00
2418,AD_CrossSpecies,2022-01-01,1,1,0.02
2419,ADMC_ADNI2-GO,2022-01-01,11,1,0.00
2420,AD-BXD,2022-01-01,1,1,0.00


In [6]:
total_files = project_summary_df.loc[0,'TOTAL_FILES']
total_data_volume = project_summary_df.loc[0, 'TOTAL_SIZE_IN_TB']

In [7]:
col1, col2, col3 = st.columns([1, 1, 5])
col1.metric("Total Number of Files", f"{total_files}")
col2.metric("Total Data Volume", f"{total_data_volume} TB")

DeltaGenerator()

In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [6]:
grouped_df = (
    unique_users_df.groupby(["STUDY", "MONTH"])["NUMBER_OF_UNIQUE_USERS"]
    .sum()
    .reset_index()
)

In [15]:
grouped_df["MONTH"] = pd.to_datetime(grouped_df["MONTH"])

In [16]:
grouped_df.loc[grouped_df["MONTH"] == "2024-07-01"]

,STUDY,MONTH,NUMBER_OF_UNIQUE_USERS
45,ACT,2024-07-01,1
76,AD-BXD,2024-07-01,4
130,ADMC_ADNI2-GO,2024-07-01,1
187,ADMC_ADNI_UHawaiiGutMetabolites,2024-07-01,1
223,AD_CrossSpecies,2024-07-01,3
...,...,...,...
2242,iPSCMicroglia,2024-07-01,3
2258,miR155,2024-07-01,2
2322,rnaSeqReprocessing,2024-07-01,18
2388,snRNAseqAD_TREM2,2024-07-01,3


In [17]:
month = pd.to_datetime(grouped_df["MONTH"])

In [18]:
month.unique

<bound method Series.unique of 0      2022-01-01
1      2022-03-01
2      2022-04-01
3      2022-05-01
4      2022-06-01
          ...    
2417   2024-04-01
2418   2024-05-01
2419   2024-06-01
2420   2024-07-01
2421   2024-08-01
Name: MONTH, Length: 2422, dtype: datetime64[ns]>

In [19]:
recent_top_studies = (
    grouped_df.loc[grouped_df["MONTH"] == "2024-07-01"]
    .sort_values(by="NUMBER_OF_UNIQUE_USERS", ascending=False)
    .head(10)
)

In [14]:
from datetime import datetime

In [15]:

current_month = datetime.today().strftime(f"%Y-%m-01")

In [16]:
current_month

'2024-08-01'

In [49]:
recent_top_studies = (
    grouped_df.loc[grouped_df["MONTH"] == current_month]
    .sort_values(by="NUMBER_OF_UNIQUE_USERS", ascending=False)
    .head(10)
)

In [20]:
fig = go.Figure()

In [21]:
thing = zip(recent_top_studies.index, recent_top_studies["STUDY"])

In [22]:
project_1 = recent_top_studies.iloc[0,0]

In [23]:
project_1

'ROSMAP'

In [24]:
filtered_df = unique_users_df[unique_users_df["STUDY"].isin([project_1])]

In [25]:
months = pd.to_datetime(filtered_df["MONTH"])

In [26]:
counts = filtered_df["NUMBER_OF_UNIQUE_USERS"]

In [27]:
# Scatter plot for the current project
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=months,
        y=list(counts),
        mode="lines+markers",
        name=project_1,
        line=dict(width=2),
        opacity=0.6,
        hoverinfo="x+y+name",
        hovertemplate="<b>Date</b>: %{x}<br><b>Users</b>: %{y}<extra></extra>",
        showlegend=True,
        visible="legendonly",
    )
)

In [30]:
total_monthly_counts = (
    unique_users_df.groupby("MONTH")["NUMBER_OF_UNIQUE_USERS"]
    .sum()
    .reset_index()
)

total_monthly_counts["MONTH"] = pd.to_datetime(total_monthly_counts["MONTH"])

In [32]:
fig.add_trace(
    go.Scatter(
        x=total_monthly_counts["MONTH"],
        y=total_monthly_counts["NUMBER_OF_UNIQUE_USERS"],
        mode="lines+markers",
        name="All Studies",
        line=dict(color="black", width=4),
        hoverinfo="x+y+name",
        hovertemplate="<b>Date</b>: %{x}<br><b>Users</b>: %{y}<extra></extra>",
        visible=True,
    )
)

In [37]:
    fig.update_layout(
        xaxis_title="Project Name",
        yaxis_title="Download Size (GB)",
        title="Download Size from Unique User Downloads (Ordered)"
    )

In [7]:
grouped_df = (
unique_users_df.groupby(["STUDY", "MONTH"])["NUMBER_OF_UNIQUE_USERS"]
.sum()
.reset_index()
)



In [8]:
grouped_df

,STUDY,MONTH,NUMBER_OF_UNIQUE_USERS
0,ACOM,2022-01-01,1
1,ACOM,2022-03-01,3
2,ACOM,2022-04-01,1
3,ACOM,2022-05-01,2
4,ACOM,2022-06-01,1
...,...,...,...
2417,snRNAseqPFC_BA10,2024-04-01,25
2418,snRNAseqPFC_BA10,2024-05-01,24
2419,snRNAseqPFC_BA10,2024-06-01,10
2420,snRNAseqPFC_BA10,2024-07-01,16


In [11]:

# Sort studies by most unique users for current month
current_month = datetime.date.today().strftime(f"%Y-%m-01")


NameError: name 'datetime' is not defined

In [12]:

datetime.today()


NameError: name 'datetime' is not defined

In [ ]:


    recent_top_studies = (
    grouped_df.loc[grouped_df["MONTH"] == current_month]
    .sort_values(by="NUMBER_OF_UNIQUE_USERS", ascending=False)
    .head(10)
    )